In [1]:
import numpy as np
import pandas as pd
import pickle 
file1 = open('X_time_2', 'rb')
file2 = open('Y_time_2', 'rb')
train_X=pickle.load(file1)
file1.close()
train_Y=pickle.load(file2)
file2.close()

In [2]:
X=train_X[:870,:]
Y=train_Y[:870]

In [3]:
import gpflow as gpf
import tensorflow as tf

from gpflow.utilities import print_summary
from gpflow.ci_utils import ci_niter


MAXITER = ci_niter(1000)
k1=gpf.kernels.SquaredExponential(lengthscales=[8.271673268662443,0.2738837077997475,1.0708216025039767,2.3060161671301564,3.1140119758334515],variance=3.4954726426406557,active_dims=[0,1,2,3,4]) 
k2=gpf.kernels.Matern32(lengthscales= 0.22953761228212077,variance=3.495472625794751,active_dims=[5])
k=k1*k2

C:\Users\Yuzhang Ge\anaconda3\envs\newenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
data=[X,Y]
m = gpf.models.GPR(data, kernel=k, mean_function=None)

In [5]:
import tensorflow as tf
optimizer = tf.optimizers.Adam(0.1)
for i in range(1000):
    optimizer.minimize(m.training_loss,var_list= m.trainable_variables)
    # likelihood = m.log_marginal_likelihood()
    # tf.print(f"GPR with Adam: iteration {i + 1} likelihood {likelihood:.04f}")

In [6]:
print_summary(m)

╒════════════════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤═════════════════════════════════════════╕
│ name                               │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │ value                                   │
╞════════════════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪═════════════════════════════════════════╡
│ GPR.kernel.kernels[0].variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 6.654769477891924                       │
├────────────────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼─────────────────────────────────────────┤
│ GPR.kernel.kernels[0].lengthscales │ Parameter │ Softplus         │         │ True        │ (5,)    │ float64 │ [6.33902592, 0.58283949, 13.33258808... │
├────────────────────────────────────┼───────────┼──────────────

In [11]:
m.training_loss

<bound method InternalDataTrainingLossMixin.training_loss of <gpflow.models.gpr.GPR object at 0x0000017960FAD6D0>>

In [31]:
m.kernel.kernels[0].lengthscales

<Parameter: name=softplus, dtype=float64, shape=[5], fn="softplus", numpy=array([ 6.33902592,  0.58283949, 13.33258808,  2.06577564,  2.13873601])>

In [22]:
tx=train_X[870:]
ty=train_Y[870:]

mean,var=m.predict_y(tx)

In [23]:
from sklearn.metrics import r2_score
print(r2_score(ty,mean))

0.8204116656701468
